In [1]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Load BLIP model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torc

In [5]:
# Load an image
image_url = "https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")

outputs = model.generate(**inputs)
caption = processor.decode(outputs[0], skip_special_tokens=True)

caption

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'organic organic tea - organic organic teas'

In [13]:
from torch.utils.data import Dataset, DataLoader

class CustomImageCaptionDataset(Dataset):
    def __init__(self, image_paths, group_ids, entity_name, captions,):
        self.image_paths = image_paths
        self.captions = captions
        self.entity_name = entity_name
        self.group_ids = group_ids
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        get_from_net = requests.get(self.image_paths[idx], stream=True).raw
        image = Image.open(get_from_net)
        # inputs = self.processor(images=image, return_tensors="pt")
        caption = self.captions[idx]
        group_id = self.group_ids[idx]
        entity_name = self.entity_name[idx]
        return inputs, group_id, entity_name, caption, group_id

In [14]:
import os
import pandas as pd

DATASET_FOLDER = '../dataset/'
train = pd.read_csv(os.path.join(DATASET_FOLDER, 'train.csv'))
test = pd.read_csv(os.path.join(DATASET_FOLDER, 'test.csv'))
sample_test = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test.csv'))
sample_test_out = pd.read_csv(os.path.join(DATASET_FOLDER, 'sample_test_out.csv'))

In [15]:
sample_train = train.iloc[:80]
sample_train

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram
...,...,...,...,...
75,https://m.media-amazon.com/images/I/71l0M0tMGj...,365637,item_weight,200.0 gram
76,https://m.media-amazon.com/images/I/71Lpqdrpi4...,365637,item_weight,200.0 gram
77,https://m.media-amazon.com/images/I/71jLIbCcwO...,365637,item_weight,100.0 gram
78,https://m.media-amazon.com/images/I/718EdwGgyV...,365637,item_weight,200.0 gram


In [26]:
num = 32
sample_train['image_link'][num] , sample_train['entity_value'][num] 

('https://m.media-amazon.com/images/I/71rKXZJrh4L.jpg', '18.55 gram')

In [18]:
dataset = CustomImageCaptionDataset(sample_train['image_link'], sample_train['group_id'], sample_train['entity_name'], sample_train['entity_value'])
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)